In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import tensorflow as tf
import json, gensim, sklearn, pickle, sys, re, os
import IPython.display as ipd

In [2]:
from gensim.parsing.preprocessing import preprocess_documents
from gensim.parsing.preprocessing import preprocess_string
from gensim.models.phrases import Phrases
from gensim.parsing.preprocessing import strip_tags, strip_short, strip_multiple_whitespaces, stem_text
from gensim.parsing.preprocessing import remove_stopwords
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim import utils
from scholarly import scholarly
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from IPython.display import Audio
sound_file = './Music/invalid_keypress.mp3'

In [3]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:
glove_file = datapath('/home/ubuntu/project/GPT2/abstracts_gpt2_medium.txt')
tmp_file = get_tmpfile("/home/ubuntu/project/GPT2/abstracts_gpt2_medium_gensim.txt")
_ = glove2word2vec(glove_file, tmp_file)
g.close()
sentence_model_e2 = KeyedVectors.load_word2vec_format(tmp_file,)

TF-IDF preparation. We will use it to provide a cutoff 

In [ ]:
datapd=pd.read_json("/home/ubuntu/project/arxiv-metadata-oai-snapshot.json",  lines=True)
datapd_hep_th = datapd.loc[datapd['categories'].str.contains('hep-th')]
hep_abstracts = datapd_hep_th[ list(datapd_hep_th.iloc[:,0:1]) + ['abstract'] + ['title'] + ['authors']]

In [ ]:
hep_corpus=hep_abstracts['abstract'].values
processed_hep=preprocess_documents(hep_corpus)
hep_dictionary = gensim.corpora.Dictionary(processed_hep)
hep_bow_corpus = [hep_dictionary.doc2bow(text) for text in processed_hep]

In [ ]:
hep_tfidf = gensim.models.TfidfModel(hep_bow_corpus, smartirs='npu')
index = gensim.similarities.MatrixSimilarity(hep_tfidf[hep_bow_corpus])

### Define a function to find most_similar to a given arxiv ID. We also use a cutoff TF-IDF score of (0.2) to remove spurious recommendations ###

In [ ]:
tij=hep_abstracts
tij = tij.reset_index(drop=True)
def fastsimplerecommend(string,string1):
    pointer3=hep_abstracts.loc[(hep_abstracts['id'].str.match(string))]
    queryvalue3=pointer3['abstract'].values
    processed_query3=preprocess_documents(queryvalue3)
    test_bow_corpus3 = [hep_dictionary.doc2bow(text) for text in processed_query3]
    vec_test_bow3=hep_tfidf[test_bow_corpus3]
    sims = index[vec_test_bow3[0]]
    ind=tij[tij['id'] == string1].index[0]
    if sims[ind]>0.2:
        return (1,sims[ind])
    return (0,0) 

def fast_gpt_tfidf_recommender(model,orig):
    i=1
    j=0
    starter=model.similar_by_word(orig,topn=1000)
    while i<31:
        x=starter[j][0]
        if fastsimplerecommend(orig,x)[0]==1:
            print(f"\n {i}) With TFIDF score {fastsimplerecommend(orig,x)[1]} and GPT-2 rank {j} ")
            print(f" Paper title : {hep_abstracts.loc[hep_abstracts['id'] == x].title.values}. \n Abstract : {hep_abstracts.loc[hep_abstracts['id'] == x].abstract.values}. \n Authors : {hep_abstracts.loc[hep_abstracts['id'] == x].authors.values}. \n Arxiv ID : {hep_abstracts.loc[hep_abstracts['id'] == x].id.values} \n \n ")
            i=i+1
        j=j+1

In [ ]:
def recommendgpt(model,string,num):
    for i in range(num):
        x=model.similar_by_word(string,topn=num)[i][0]
        print(f"{i+1}) Paper title : {hep_abstracts.loc[hep_abstracts['id'] == x].title.values}. \n Abstract : {hep_abstracts.loc[hep_abstracts['id'] == x].abstract.values}. \n Authors : {hep_abstracts.loc[hep_abstracts['id'] == x].authors.values}. \n Arxiv ID : {hep_abstracts.loc[hep_abstracts['id'] == x].id.values} \n \n ")
        i=i+1

In [11]:
recommendgpt(sentence_model_e2,"1212.3616",30)

1) Paper title : ['Bounding scalar operator dimensions in 4D CFT']. 
 Abstract : ['  In an arbitrary unitary 4D CFT we consider a scalar operator \\phi, and the\noperator \\phi^2 defined as the lowest dimension scalar which appears in the OPE\n\\phi\\times\\phi with a nonzero coefficient. Using general considerations of OPE,\nconformal block decomposition, and crossing symmetry, we derive a\ntheory-independent inequality [\\phi^2] \\leq f([\\phi]) for the dimensions of\nthese two operators. The function f(d) entering this bound is computed\nnumerically. For d->1 we have f(d)=2+O(\\sqrt{d-1}), which shows that the free\ntheory limit is approached continuously. We perform some checks of our bound.\nWe find that the bound is satisfied by all weakly coupled 4D conformal fixed\npoints that we are able to construct. The Wilson-Fischer fixed points violate\nthe bound by a constant O(1) factor, which must be due to the subtleties of\nextrapolating to 4-\\epsilon dimensions. We use our method t

7) Paper title : ['Minkowski Conformal Blocks and the Regge Limit for SYK-like Models']. 
 Abstract : ['  We discuss scattering in a CFT via the conformal partial-wave analysis and\nthe Regge limit. The focus of this paper is on understanding an OPE with\nMinkowski conformal blocks. Starting with a t-channel OPE, it leads to an\nexpansion for an s-channel scattering amplitude in terms of t-channel\nexchanges. By contrasting with Euclidean conformal blocks we see a precise\nrelationship between conformal blocks in the two limits without preforming an\nexplicit analytic continuation. We discuss a generic feature for a CFT\ncorrelation function having singular $F^{(M)}(u,v)\\sim {u}^{-\\delta}\\,$,\n$\\delta>0$, in the limit $u \\rightarrow 0$ and $v\\rightarrow 1$. Here,\n$\\delta=(\\ell_{eff}-1)/2$, with $\\ell_{eff}$ serving as an effective spin and\nit can be determined through an OPE. In particular, it is bounded from above,\n$\\ell_{eff} \\leq 2$, for all CFTs with a gravity dual, a

13) Paper title : ['Bounds on corner entanglement in quantum critical states']. 
 Abstract : ["  The entanglement entropy in many gapless quantum systems receives a\ncontribution from corners in the entangling surface in 2+1d. It is\ncharacterized by a universal function $a(\\theta)$ depending on the opening\nangle $\\theta$, and contains pertinent low energy information. For conformal\nfield theories (CFTs), the leading expansion coefficient in the smooth limit\n$\\theta \\to \\pi$ yields the stress tensor 2-point function coefficient $C_T$ .\nLittle is known about $a(\\theta)$ beyond that limit. Here, we show that the\nnext term in the smooth limit expansion contains information beyond the 2- and\n3-point correlators of the stress tensor. We conjecture that it encodes 4-point\ndata, making it much richer. Further, we establish strong constraints on this\nand higher order smooth-limit coefficients. We also show that $a(\\theta)$ is\nlower-bounded by a non-trivial function multiplied b

19) Paper title : ['The inside outs of AdS(3)/CFT(2): Exact AdS wormholes with entangled CFT\n  duals']. 
 Abstract : ["  We present the complete family of solutions of 3D gravity (Lambda<0) with two\nasymptotically AdS exterior regions. The solutions are constructed from data at\nthe two boundaries, which correspond to two independent and arbitrary stress\ntensors T_R, \\bar T_R, and T_L, \\bar T_L. The two exteriors are smoothly joined\non to an interior region through a regular horizon. We find CFT duals of these\ngeometries which are entangled states of two CFT's. We compute correlators\nbetween general operators at the two boundaries and find perfect agreement\nbetween CFT and bulk calculations. We calculate and match the CFT entanglement\nentropy (EE) with the holographic EE which involves geodesics passing through\nthe wormhole. We also compute a holographic, non-equilibrium entropy for the\nCFT using properties of the regular horizon. The construction of the bulk\nsolutions her

25) Paper title : ['Topologically Massive Gravity and the AdS/CFT Correspondence']. 
 Abstract : ["  We set up the AdS/CFT correspondence for topologically massive gravity (TMG)\nin three dimensions. The first step in this procedure is to determine the\nappropriate fall off conditions at infinity. These cannot be fixed a priori as\nthey depend on the bulk theory under consideration and are derived by solving\nasymptotically the non-linear field equations. We discuss in detail the\nasymptotic structure of the field equations for TMG, showing that it contains\nleading and subleading logarithms, determine the map between bulk fields and\nCFT operators, obtain the appropriate counterterms needed for holographic\nrenormalization and compute holographically one- and two-point functions at and\naway from the 'chiral point' (mu = 1). The 2-point functions at the chiral\npoint are those of a logarithmic CFT (LCFT) with c_L = 0, c_R = 3l/G_N and b =\n-3l/G_N, where b is a parameter characterizin

In [14]:
recommendgpt(sentence_model_e2,"1212.3616",30)

1) Paper title : ['Bounding scalar operator dimensions in 4D CFT']. 
 Abstract : ['  In an arbitrary unitary 4D CFT we consider a scalar operator \\phi, and the\noperator \\phi^2 defined as the lowest dimension scalar which appears in the OPE\n\\phi\\times\\phi with a nonzero coefficient. Using general considerations of OPE,\nconformal block decomposition, and crossing symmetry, we derive a\ntheory-independent inequality [\\phi^2] \\leq f([\\phi]) for the dimensions of\nthese two operators. The function f(d) entering this bound is computed\nnumerically. For d->1 we have f(d)=2+O(\\sqrt{d-1}), which shows that the free\ntheory limit is approached continuously. We perform some checks of our bound.\nWe find that the bound is satisfied by all weakly coupled 4D conformal fixed\npoints that we are able to construct. The Wilson-Fischer fixed points violate\nthe bound by a constant O(1) factor, which must be due to the subtleties of\nextrapolating to 4-\\epsilon dimensions. We use our method t

7) Paper title : ['AdS Description of Induced Higher-Spin Gauge Theory']. 
 Abstract : ["  We study deformations of three-dimensional large N CFTs by double-trace\noperators constructed from spin s single-trace operators of dimension \\Delta.\nThese theories possess UV fixed points, and we calculate the change of the\n3-sphere free energy \\delta F= F_{UV}- F_{IR}. To describe the UV fixed point\nusing the dual AdS_4 space we modify the boundary conditions on the spin s\nfield in the bulk; this approach produces \\delta F in agreement with the field\ntheory calculations. If the spin s operator is a conserved current, then the\nfixed point is described by an induced parity invariant conformal spin s gauge\ntheory. The low spin examples are QED_3 (s=1) and the 3-d induced conformal\ngravity (s=2). When the original CFT is that of N conformal complex scalar or\nfermion fields, the U(N) singlet sector of the induced 3-d gauge theory is dual\nto Vasiliev's theory in AdS_4 with alternate bou

13) Paper title : ['Structure of large spin expansion of anomalous dimensions at strong\n  coupling']. 
 Abstract : ['  The anomalous dimensions of planar N=4 SYM theory operators like tr(Phi D^S\nPhi) expanded in large spin S have the asymptotics \\gamma= f ln S + f_c + 1/S\n(f_11 ln S + f_10) + ..., where f (the universal scaling function or cusp\nanomaly), f_c and f_mn are given by power series in the `t Hooft coupling\n\\lambda. The subleading coefficients appear to be related by the so called\nfunctional relation and parity invariance (or reciprocity) property of the\nfunction expressing \\gamma in terms of the conformal spin of the collinear\ngroup. Here we study the structure of such large spin expansion at strong\ncoupling via AdS/CFT, i.e. by using the dual description in terms of folded\nspinning string in AdS_5. The large spin expansion of the classical string\nenergy happens to have the same structure as that of \\gamma in the perturbative\ngauge theory. Moreover, the funct

19) Paper title : ['Bounds on SCFTs from Conformal Perturbation Theory']. 
 Abstract : ['  The operator product expansion (OPE) in 4d (super)conformal field theory is\nof broad interest, for both formal and phenomenological applications. In this\npaper, we use conformal perturbation theory to study the OPE of nearly-free\nfields coupled to SCFTs. Under fairly general assumptions, we show that the OPE\nof a chiral operator of dimension $\\Delta = 1+\\epsilon$ with its complex\nconjugate always contains an operator of dimension less than $2 \\Delta$. Our\nbounds apply to Banks-Zaks fixed points and their generalizations, as we\nillustrate using several examples.\n']. 
 Authors : ['Daniel Green and David Shih']. 
 Arxiv ID : ['1203.5129'] 
 
 
20) Paper title : ['Convexity and Liberation at Large Spin']. 
 Abstract : ['  We consider several aspects of unitary higher-dimensional conformal field\ntheories (CFTs). We first study massive deformations that trigger a flow to a\ngapped phase. De

25) Paper title : ['Topologically Massive Gravity and the AdS/CFT Correspondence']. 
 Abstract : ["  We set up the AdS/CFT correspondence for topologically massive gravity (TMG)\nin three dimensions. The first step in this procedure is to determine the\nappropriate fall off conditions at infinity. These cannot be fixed a priori as\nthey depend on the bulk theory under consideration and are derived by solving\nasymptotically the non-linear field equations. We discuss in detail the\nasymptotic structure of the field equations for TMG, showing that it contains\nleading and subleading logarithms, determine the map between bulk fields and\nCFT operators, obtain the appropriate counterterms needed for holographic\nrenormalization and compute holographically one- and two-point functions at and\naway from the 'chiral point' (mu = 1). The 2-point functions at the chiral\npoint are those of a logarithmic CFT (LCFT) with c_L = 0, c_R = 3l/G_N and b =\n-3l/G_N, where b is a parameter characterizin

In [14]:
fast_gpt_tfidf_recommender(sentence_model_e2,"1212.3616")


 1) With TFIDF score 0.4061584770679474 and GPT-2 rank 0 
 Paper title : ['Bounding scalar operator dimensions in 4D CFT']. 
 Abstract : ['  In an arbitrary unitary 4D CFT we consider a scalar operator \\phi, and the\noperator \\phi^2 defined as the lowest dimension scalar which appears in the OPE\n\\phi\\times\\phi with a nonzero coefficient. Using general considerations of OPE,\nconformal block decomposition, and crossing symmetry, we derive a\ntheory-independent inequality [\\phi^2] \\leq f([\\phi]) for the dimensions of\nthese two operators. The function f(d) entering this bound is computed\nnumerically. For d->1 we have f(d)=2+O(\\sqrt{d-1}), which shows that the free\ntheory limit is approached continuously. We perform some checks of our bound.\nWe find that the bound is satisfied by all weakly coupled 4D conformal fixed\npoints that we are able to construct. The Wilson-Fischer fixed points violate\nthe bound by a constant O(1) factor, which must be due to the subtleties of\next


 7) With TFIDF score 0.29217904806137085 and GPT-2 rank 6 
 Paper title : ['Minkowski Conformal Blocks and the Regge Limit for SYK-like Models']. 
 Abstract : ['  We discuss scattering in a CFT via the conformal partial-wave analysis and\nthe Regge limit. The focus of this paper is on understanding an OPE with\nMinkowski conformal blocks. Starting with a t-channel OPE, it leads to an\nexpansion for an s-channel scattering amplitude in terms of t-channel\nexchanges. By contrasting with Euclidean conformal blocks we see a precise\nrelationship between conformal blocks in the two limits without preforming an\nexplicit analytic continuation. We discuss a generic feature for a CFT\ncorrelation function having singular $F^{(M)}(u,v)\\sim {u}^{-\\delta}\\,$,\n$\\delta>0$, in the limit $u \\rightarrow 0$ and $v\\rightarrow 1$. Here,\n$\\delta=(\\ell_{eff}-1)/2$, with $\\ell_{eff}$ serving as an effective spin and\nit can be determined through an OPE. In particular, it is bounded from above,\n


 13) With TFIDF score 0.3071422874927521 and GPT-2 rank 46 
 Paper title : ['More ${\\mathcal N}=4$ superconformal bootstrap']. 
 Abstract : ['  In this long overdue second installment, we continue to develop the conformal\nbootstrap program for ${\\mathcal N}=4$ superconformal field theories in four\ndimensions via an analysis of the correlation function of four stress-tensor\nsupermultiplets. We review analytic results for this correlator and make\ncontact with the SCFT/chiral algebra correspondence of arXiv:1312.5344. We\ndemonstrate that the constraints of unitarity and crossing symmetry require the\ncentral charge $c$ to be greater than or equal to $3/4$ in any interacting\n${\\mathcal N}=4$ SCFT. We apply numerical bootstrap methods to derive upper\nbounds on scaling dimensions and OPE coefficients for several low-lying,\nunprotected operators as a function of the central charge. We interpret our\nbounds in the context of ${\\mathcal N}=4$ super Yang-Mills (SYM) theories,\nformu


 18) With TFIDF score 0.23080967366695404 and GPT-2 rank 60 
 Paper title : ['Universality in asymptotic bounds and its saturation in $2$D CFT']. 
 Abstract : ['  We study asymptotics of three point coefficients (light-light-heavy) and two\npoint correlators in heavy states in unitary, compact $2$D CFTs. We prove an\nupper and lower bound on such quantities using numerically assisted Tauberian\ntechniques. We obtain an optimal upper bound on the spectrum of operators\nappearing with fixed spin from the OPE of two identical scalars. While all the\nCFTs obey this bound, rational CFTs come close to saturating it. This mimics\nthe scenario of bounds on asymptotic density of states and thereby pronounces\nan universal feature in asymptotics of 2D CFTs. Next, we clarify the role of\nsmearing in interpreting the asymptotic results pertaining to considerations of\neigenstate thermalization in 2D CFTs. In the context of light-light-heavy three\npoint coefficients, we find that the order one nu


 24) With TFIDF score 0.30315929651260376 and GPT-2 rank 111 
 Paper title : ['Conformal Regge theory']. 
 Abstract : ["  We generalize Regge theory to correlation functions in conformal field\ntheories. This is done by exploring the analogy between Mellin amplitudes in\nAdS/CFT and S-matrix elements. In the process, we develop the conformal partial\nwave expansion in Mellin space, elucidating the analytic structure of the\npartial amplitudes. We apply the new formalism to the case of four point\ncorrelation functions between protected scalar operators in N=4 Super Yang\nMills, in cases where the Regge limit is controlled by the leading twist\noperators associated to the pomeron-graviton Regge trajectory. At weak\ncoupling, we are able to predict to arbitrary high order in the 't Hooft\ncoupling the behaviour near J=1 of the OPE coefficients C_{OOJ} between the\nexternal scalars and the spin J leading twist operators. At strong coupling, we\nuse recent results for the anomalous dimens


 29) With TFIDF score 0.47028008103370667 and GPT-2 rank 152 
 Paper title : ['Bounds on 4D Conformal and Superconformal Field Theories']. 
 Abstract : ['  We derive general bounds on operator dimensions, central charges, and OPE\ncoefficients in 4D conformal and N=1 superconformal field theories. In any CFT\ncontaining a scalar primary phi of dimension d we show that crossing symmetry\nof <phi phi phi phi> implies a completely general lower bound on the central\ncharge c >= f_c(d). Similarly, in CFTs containing a complex scalar charged\nunder global symmetries, we bound a combination of symmetry current two-point\nfunction coefficients tau^{IJ} and flavor charges. We extend these bounds to\nN=1 superconformal theories by deriving the superconformal block expansions for\nfour-point functions of a chiral superfield Phi and its conjugate. In this case\nwe derive bounds on the OPE coefficients of scalar operators appearing in the\nPhi x Phi* OPE, and show that there is an upper bound on 

In [17]:
recommendgpt(sentence_model_e2,"1908.10996",30)

1) Paper title : ['Geometric secret sharing in a model of Hawking radiation']. 
 Abstract : ['  We consider a black hole in three dimensional AdS space entangled with an\nauxiliary radiation system. We model the microstates of the black hole in terms\nof a field theory living on an end of the world brane behind the horizon, and\nallow this field theory to itself have a holographic dual geometry. This\ngeometry is also a black hole since entanglement of the microstates with the\nradiation leaves them in a mixed state. This "inception black hole" can be\npurified by entanglement through a wormhole with an auxiliary system which is\nnaturally identified with the external radiation, giving a realization of the\nER=EPR scenario. In this context, we propose an extension of the Ryu-Takayanagi\n(RT) formula, in which extremal surfaces computing entanglement entropy are\nallowed to pass through the brane into its dual geometry. This new rule\nreproduces the Page curve for evaporating black hole

7) Paper title : ['Effective entropy of quantum fields coupled with gravity']. 
 Abstract : ['  Entanglement entropy quantifies the amount of uncertainty of a quantum state.\nFor quantum fields in curved space, entanglement entropy of the quantum field\ntheory degrees of freedom is well-defined for a fixed background geometry. In\nthis paper, we propose a generalization of the quantum field theory\nentanglement entropy by including dynamical gravity. The generalized quantity\nnamed effective entropy, and its Renyi entropy generalizations, are defined by\nanalytic continuation of a gravitational path integral on replica geometry with\na co-dimension-$2$ brane at the boundary of region we are studying. We discuss\ndifferent approaches to define the region in a gauge invariant way, and show\nthat the effective entropy satisfies the quantum extremal surface formula. When\nthe quantum fields carry a significant amount of entanglement, the quantum\nextremal surface can have a topology transi

12) Paper title : ['Do We Understand Black Hole Entropy ?']. 
 Abstract : ["  I review various proposals for the nature of black hole entropy and for the\nmechanism behind the operation of the generalized second law. I stress the\nmerits of entanglement entropy {\\tenit qua\\/} black hole entropy, and point out\nthat, from an operational viewpoint, entanglement entropy is perfectly finite.\nProblems with this identification such as the multispecies problem and the\ntrivialization of the information puzzle are mentioned. This last leads me to\nassociate black hole entropy rather with the multiplicity of density operators\nwhich describe a black hole according to exterior observers. I relate this\nidentification to Sorkin's proof of the generalized second law. I discuss in\nsome depth Frolov and Page's proof of the same law, finding it relevant only\nfor scattering of microsystems by a black hole. Assuming that the law is\ngenerally valid I make evident the existence of the universal bou

18) Paper title : ['Holographic entanglement entropy of semi-local quantum liquids']. 
 Abstract : ['  We consider the holographic entanglement entropy of $(d+2)$-dimensional\nsemi-local quantum liquids, for which the dual gravity background in the deep\ninterior is $AdS_{2}\\times\\mathbb{R}^{d}$ multiplied by a warp factor which\ndepends on the radial coordinate. The entropy density of this geometry goes to\nzero in the extremal limit. The thermodynamics associated with this semi-local\nbackground is discussed via dimensional analysis and scaling arguments. For the\ncase of an asymptotically AdS UV completion of this geometry, we show that the\nentanglement entropy of a strip and an annulus exhibits a phase transition as a\ntypical length of the different shapes is varied, while there is no sign of\nsuch a transition for the entanglement entropy of a sphere. Moreover, for the\nspherical entangling region, the leading order contribution to the entanglement\nentropy in the IR is calcul

24) Paper title : ['Renormalization group flow of entanglement entropy on spheres']. 
 Abstract : ["  We explore entanglement entropy of a cap-like region for a generic quantum\nfield theory residing in the Bunch-Davies vacuum on de Sitter space.\nEntanglement entropy in our setup is identical with the thermal entropy in the\nstatic patch of de Sitter, and we derive a simple relation between the vacuum\nexpectation value of the energy-momentum tensor trace and the RG flow of\nentanglement entropy. In particular, renormalization of the cosmological\nconstant and logarithmic divergence of the entanglement entropy are\ninterrelated in our setup. We confirm our findings by recovering known\nuniversal contributions for a free field theory deformed by a mass operator as\nwell as obtain correct universal behaviour at the fixed points. Simple examples\nof entanglement entropy flows are elaborated in $d=2,3,4$. In three dimensions\nwe find that while the renormalized entanglement entropy is sta

30) Paper title : ['Evaporating Black Holes Coupled to a Thermal Bath']. 
 Abstract : ['  We study the doubly holographic model of [arXiv:1908.10996] in the situation\nwhere a black hole in two-dimensional JT gravity theory is coupled to an\nauxiliary bath system at arbitrary finite temperature. Depending on the initial\ntemperature of the black hole relative to the bath temperature, the black hole\ncan lose mass by emitting Hawking radiation, stay in equilibrium with the bath\nor gain mass by absorbing thermal radiation from the bath. In all of these\nscenarios, a unitary Page curve is obtained by applying the usual prescription\nfor holographic entanglement entropy and identifying the quantum extremal\nsurface for the generalized entropy, using both analytical and numeric\ncalculations. As the application of the entanglement wedge reconstruction, we\nfurther investigate the reconstruction of the black hole interior from a\nsubsystem containing the Hawking radiation. We examine the ro

In [16]:
fast_gpt_tfidf_recommender(sentence_model_e2,"1908.10996")


 1) With TFIDF score 0.563462495803833 and GPT-2 rank 0 
 Paper title : ['Geometric secret sharing in a model of Hawking radiation']. 
 Abstract : ['  We consider a black hole in three dimensional AdS space entangled with an\nauxiliary radiation system. We model the microstates of the black hole in terms\nof a field theory living on an end of the world brane behind the horizon, and\nallow this field theory to itself have a holographic dual geometry. This\ngeometry is also a black hole since entanglement of the microstates with the\nradiation leaves them in a mixed state. This "inception black hole" can be\npurified by entanglement through a wormhole with an auxiliary system which is\nnaturally identified with the external radiation, giving a realization of the\nER=EPR scenario. In this context, we propose an extension of the Ryu-Takayanagi\n(RT) formula, in which extremal surfaces computing entanglement entropy are\nallowed to pass through the brane into its dual geometry. This new ru


 6) With TFIDF score 0.3986660838127136 and GPT-2 rank 5 
 Paper title : ['Entanglement Wedge Reconstruction and the Information Paradox']. 
 Abstract : ['  When absorbing boundary conditions are used to evaporate a black hole in\nAdS/CFT, we show that there is a phase transition in the location of the\nquantum Ryu-Takayanagi surface, at precisely the Page time. The new RT surface\nlies slightly inside the event horizon, at an infalling time approximately the\nscrambling time $\\beta/2\\pi \\log S_{BH}$ into the past. We can immediately\nderive the Page curve, using the Ryu-Takayanagi formula, and the\nHayden-Preskill decoding criterion, using entanglement wedge reconstruction.\nBecause part of the interior is now encoded in the early Hawking radiation, the\ndecreasing entanglement entropy of the black hole is exactly consistent with\nthe semiclassical bulk entanglement of the late-time Hawking modes, despite the\nabsence of a firewall.\n  By studying the entanglement wedge of highly 


 11) With TFIDF score 0.272577702999115 and GPT-2 rank 10 
 Paper title : ['Black hole entropy and the renormalization group']. 
 Abstract : ['  Four decades after its first postulation by Bekenstein, black hole entropy\nremains mysterious. It has long been suggested that the entanglement entropy of\nquantum fields on the black hole gravitational background should represent at\nleast an important contribution to the total Bekenstein-Hawking entropy, and\nthat the divergences in the entanglement entropy should be absorbed in the\nrenormalization of the gravitational couplings. In this talk, we describe how\nan improved understanding of black hole entropy is obtained by combining these\nnotions with the renormalization group. By introducing an RG flow scale, we\ninvestigate whether the total entropy of the black hole can be partitioned in a\n"gravitational" part related to the flowing gravitational action, and a\n"quantum" part related to the unintegrated degrees of freedom. We describe


 17) With TFIDF score 0.221995547413826 and GPT-2 rank 16 
 Paper title : ['Holographic Evolution of Entanglement Entropy']. 
 Abstract : ['  We study the evolution of entanglement entropy in a 2-dimensional\nequilibration process that has a holographic description in terms of a Vaidya\ngeometry. It models a unitary evolution in which the field theory starts in a\npure state, its vacuum, and undergoes a perturbation that brings it far from\nequilibrium. The entanglement entropy in this set up provides a measurement of\nthe quantum entanglement in the system. Using holographic techniques we recover\nthe same result obtained before from the study of processes triggered by a\nsudden change in a parameter of the hamiltonian, known as quantum quenches.\nNamely, entanglement in 2-dimensional conformal field theories propagates with\nvelocity v^2=1. Both in quantum quenches and in the Vaidya model equilibration\nis only achieved at the local level. Remarkably, the holographic derivation of\n


 23) With TFIDF score 0.3419891595840454 and GPT-2 rank 22 
 Paper title : ['Islands in de Sitter space']. 
 Abstract : ['  We consider black holes in 2d de Sitter JT gravity coupled to a CFT, and\nentangled with matter in a disjoint non-gravitating universe. Tracing out the\nentangling matter leaves the CFT in a density matrix whose stress tensor\nbackreacts on the de Sitter geometry, lengthening the wormhole behind the black\nhole horizon. Naively, the entropy of the entangling matter increases without\nbound as the strength of the entanglement increases, but the monogamy property\npredicts that this growth must level off. We compute the entropy via the\nreplica trick, including wormholes between the replica copies of the de Sitter\ngeometry, and find a competition between conventional field theory entanglement\nentropy and the surface area of extremal "islands" in the de Sitter geometry.\nThe black hole and cosmological horizons both play a role in generating such\nislands in the b


 29) With TFIDF score 0.2539184093475342 and GPT-2 rank 28 
 Paper title : ['Entanglement dynamics after quantum quenches in generic integrable\n  systems']. 
 Abstract : ['  The time evolution of the entanglement entropy in non-equilibrium quantum\nsystems provides crucial information about the structure of the time-dependent\nstate. For quantum quench protocols, by combining a quasiparticle picture for\nthe entanglement spreading with the exact knowledge of the stationary state\nprovided by Bethe ansatz, it is possible to obtain an exact and analytic\ndescription of the evolution of the entanglement entropy. Here we discuss the\napplication of these ideas to several integrable models. First we show that for\nnon-interacting systems, both bosonic and fermionic, the exact time-dependence\nof the entanglement entropy can be derived by elementary techniques and without\nsolving the dynamics. We then provide exact results for interacting spin chains\nthat are carefully tested against num

In [60]:
recommend("1908.00015",25) #pure tf-idf results

 
 1) A note on defect Mellin amplitudes by Vasco Goncalves, Georgios Itsios (0.65971667). Arxiv: 1803.06721
 
 2) The Next $16$ Higher Spin Currents and Three-Point Functions in the
  Large ${\cal N}=4$ Holography by Changhyun Ahn, Dong-gyu Kim and Man Hea Kim (0.6346596). Arxiv: 1703.01744
 
 3) Bootstrapping string theory on AdS$_5 \times S^5$ by J. M. Drummond, H. Paul and M. Santagata (0.63369775). Arxiv: 2004.07282
 
 4) Nonperturbative Mellin Amplitudes: Existence, Properties, Applications by Joao Penedones, Joao A. Silva, Alexander Zhiboedov (0.6324333). Arxiv: 1912.11100
 
 5) OPE inversion in Mellin space by Carlos Cardona (0.6264813). Arxiv: 1803.05086
 
 6) The Operator Product Expansions in the ${\cal N}=4$ Orthogonal Wolf
  Space Coset Model by Changhyun Ahn, Man Hea Kim and Jinsub Paeng (0.6253749). Arxiv: 1904.06855
 
 7) Towards Spinning Mellin Amplitudes by Heng-Yu Chen, En-Jui Kuo and Hideki Kyono (0.6166204). Arxiv: 1712.07991
 
 8) Towards a Bootstrap approach to h

In [58]:
fastlstmrecommender(sentence_model_e2,"1908.00015")


 1). With TFIDF rank 0.17074674367904663 and LSTM rank 1 
2) Paper title : ['Quantum vortices, M2-branes and black holes']. 
 Abstract : ['  We study the partition functions of BPS vortices and magnetic monopole\noperators, in gauge theories describing $N$ M2-branes. In particular, we\nexplore two closely related methods to study the Cardy limit of the index on\n$S^2\\times\\mathbb{R}$. The first method uses the factorization of this index to\nvortex partition functions, while the second one uses a continuum approximation\nfor the monopole charge sums. Monopole condensation confines most of the $N^2$\ndegrees of freedom except $N^{\\frac{3}{2}}$ of them, even in the high\ntemperature deconfined phase. The resulting large $N$ free energy statistically\naccounts for the Bekenstein-Hawking entropy of large BPS black holes in\n$AdS_4\\times S^7$. Our Cardy free energy also suggests a finite $N$ version of\nthe $N^{\\frac{3}{2}}$ degrees of freedom.\n']. 
 Authors : ['Sunjin Choi, Chiung H


 8). With TFIDF rank 0.1806628555059433 and LSTM rank 39 
9) Paper title : ['Conformally Soft Photons and Gravitons']. 
 Abstract : ['  The four-dimensional $S$-matrix is reconsidered as a correlator on the\ncelestial sphere at null infinity. Asymptotic particle states can be\ncharacterized by the point at which they enter or exit the celestial sphere as\nwell as their $SL(2,\\mathbb C)$ Lorentz quantum numbers: namely their conformal\nscaling dimension and spin $h\\pm \\bar h$ instead of the energy and momentum.\nThis characterization precludes the notion of a soft particle whose energy is\ntaken to zero. We propose it should be replaced by the notion of a conformally\nsoft particle with $h=0$ or $\\bar h=0$. For photons we explicitly construct\nconformally soft $SL(2,\\mathbb C)$ currents with dimensions $(1,0)$ and\nidentify them with the generator of a $U(1)$ Kac-Moody symmetry on the\ncelestial sphere. For gravity the generator of celestial conformal symmetry is\nconstructed from


 13). With TFIDF rank 0.1734973043203354 and LSTM rank 65 
14) Paper title : ['Quantum Information Metric on $\\mathbb{R} \\times S^{d-1}$']. 
 Abstract : ['  We present a formula for the information metric on $\\mathbb{R} \\times\n{S}^{d-1}$ for a scalar primary operator of integral dimension $\\Delta \\, (\\,\\,\n> \\frac{d+1}{2})$. This formula is checked for various space-time dimensions\n$d$ and $\\Delta$ in the field theory side. We check the formula in the gravity\nside using the holographic setup. We clarify the regularization and\nrenormalization involved in these computations. We also show that the quantum\ninformation metric of an exactly marginal operator agrees with the leading\norder of the interface free energy of the conformal Janus on Euclidean ${S}^d$,\nwhich is checked for $d=2, 3$.\n']. 
 Authors : ['Dongsu Bak and Andrea Trivella']. 
 Arxiv ID : ['1707.05366'] 
 
 

 13

 14). With TFIDF rank 0.2037607729434967 and LSTM rank 70 
15) Paper title : ['Holographic Rec


 19). With TFIDF rank 0.3393406271934509 and LSTM rank 113 
20) Paper title : ['Polyakov-Mellin Bootstrap for AdS loops']. 
 Abstract : ['  We consider holographic CFTs and study their large $N$ expansion. We use\nPolyakov-Mellin bootstrap to extract the CFT data of all operators, including\nscalars, till $O(1/N^4)$. We add a contact term in Mellin space, which\ncorresponds to an effective $\\phi^4$ theory in AdS and leads to anomalous\ndimensions for scalars at $O(1/N^2)$. Using this we fix $O(1/N^4)$ anomalous\ndimensions for double trace operators finding perfect agreement with\n\\cite{loopal} (for $\\Delta_{\\phi}=2$). Our approach generalizes this to any\ndimensions and any value of conformal dimensions of external scalar field. In\nthe second part of the paper, we compute the loop amplitude in AdS which\ncorresponds to non-planar correlators of in CFT. More precisely, using CFT data\nat $O(1/N^4)$ we fix the AdS bubble diagram and the triangle diagram for the\ngeneral case.\n'].


 25). With TFIDF rank 0.16374078392982483 and LSTM rank 187 
26) Paper title : ['A New Constraint on Effective Field Theories of the QCD Flux Tube']. 
 Abstract : ['  Effective magnetic $SU(N)$ gauge theory with classical $Z_N$ flux tubes of\nintrinsic width $\\frac{1}{M}$ is an effective field theory of the long distance\nquark-antiquark interaction in $SU(N)$ Yang-Mills theory. Long wavelength\nfluctuations of the $Z_N$ vortices of this theory lead to an effective string\ntheory. In this paper we clarify the connection between effective field theory\nand effective string theory and we propose a new constraint on these vortices.\nWe first examine the impact of string fluctuations on the classical dual\nsuperconductor description of confinement. At inter-quark distances $R\\sim\n\\frac{1}{M}$ the classical action for a straight flux tube determines the heavy\nquark potentials. At distances $R \\gg \\frac{1}{M}$ fluctuations of the flux\ntube axis $\\tilde{x}$ give rise to an effective

In [31]:
recommendgpt("1908.00015",30)

/home/ubuntu/anaconda3/envs/aws_neuron_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


0) Paper title : ['Modular Invariance in Superstring Theory From ${\\cal N} = 4$\n  Super-Yang-Mills']. 
 Abstract : ['  We study the four-point function of the lowest-lying half-BPS operators in\nthe ${\\cal N} =4$ $SU(N)$ super-Yang-Mills theory and its relation to the\nflat-space four-graviton amplitude in type IIB superstring theory. We work in a\nlarge-$N$ expansion in which the complexified Yang-Mills coupling $\\tau$ is\nfixed. In this expansion, non-perturbative instanton contributions are present,\nand the $SL(2, \\mathbb{Z})$ duality invariance of correlation functions is\nmanifest. Our results are based on a detailed analysis of the sphere partition\nfunction of the mass-deformed SYM theory, which was previously computed using\nsupersymmetric localization. This partition function determines a certain\nintegrated correlator in the undeformed ${\\cal N} = 4$ SYM theory, which in\nturn constrains the four-point correlator at separated points. In a\nnormalization where the two-p

6) Paper title : ['Platonic Field Theories']. 
 Abstract : ['  We study renormalization group (RG) fixed points of scalar field theories\nendowed with the discrete symmetry groups of regular polytopes. We employ the\nfunctional perturbative renormalization group (FPRG) approach and the\n$\\epsilon$-expansion in $d=d_c-\\epsilon$. The upper critical dimensions\nrelevant to our analysis are $d_c =\n6,4,\\frac{10}{3},3,\\frac{14}{5},\\frac{8}{3},\\frac{5}{2},\\frac{12}{5}$; in order\nto get access to the corresponding RG beta functions, we derive general\nmulticomponent beta functionals $\\beta_V$ and $\\beta_Z$ in the aforementioned\nupper critical dimensions, most of which are novel. The field theories we\nanalyze have $N=2$ (polygons), $N=3$ (Platonic solids) and $N=4$\n(hyper-Platonic solids) field components. The main results of this analysis\ninclude a new candidate universality class in three physical dimensions based\non the symmetry group $\\mathbb{D}_5$ of the Pentagon. Moreover

12) Paper title : ['Exact $\\nabla^4 R^4$ couplings and helicity supertraces']. 
 Abstract : ['  In type II string theory compactified on a $d$-dimensional torus $T^d$ down\nto $D=10-d$ dimensions, the $R^4$ and $\\nabla^4 R^4$ four-graviton couplings\nare known exactly, for all values of the moduli, in terms of certain Eisenstein\nseries of the U-duality group $E_{d}(\\mathbb{Z})$. In the limit where one\ncircle in the torus becomes large, these couplings are expected to reduce to\ntheir counterpart in dimension $D+1$, plus threshold effects and exponentially\nsuppressed corrections corresponding to BPS black holes in dimension $D+1$\nwhose worldline winds around the circle. By combining the weak coupling and\nlarge radius limits, we determine these exponentially suppressed corrections\nexactly, and demonstrate that the contributions of 1/4-BPS black holes to the\n$\\nabla^4 R^4$ coupling are proportional to the appropriate helicity\nsupertrace. Mathematically, our results provide the

20) Paper title : ['Confinement and Pseudoscalar Glueball Spectrum in the $2+1$D QCD-Like\n  Theory from the Non-Susy D$2$ Brane']. 
 Abstract : ['  Here we study two important properties of $2+1$ dimensional QCD --\nconfinement and pseudoscalar glueball spectrum -- with holographic approach. We\nconsider the low energy decoupled geometry of the isotropic non-susy D$2$\nbrane. We find the corresponding gauge theory is similar to the $2+1$-dim\nYang-Mills theory with the running coupling $\\lambda^2$. At the extremal limit\n(i.e. BPS limit), this gauge theory reduces to the super-YM theory. From the\nNambu-Goto action of a test string, the potential of a \\qq pair located on the\nboundary is calculated. At large \\qq separation it gives the tension $\\sigma$\nof the QCD flux-tube. The parametric dependencies of $\\sigma$ is shown\npictorially. It is found that $\\sigma$ is a monotonically increasing function\nof the effective coupling $\\lambda^2$. In comparison,\n$\\sqrt{\\sigma}/g_\\t

26) Paper title : ['Induced QCD II: Numerical results']. 
 Abstract : ['  We numerically explore an alternative discretization of continuum\n$\\text{SU}(N_c)$ Yang-Mills theory on a Euclidean spacetime lattice, originally\nintroduced by Budzcies and Zirnbauer for gauge group $\\text{U}(N_c)$. This\ndiscretization can be reformulated such that the self-interactions of the gauge\nfield are induced by a path integral over $N_b$ auxiliary bosonic fields, which\ncouple linearly to the gauge field. In the first paper of the series we have\nshown that the theory reproduces continuum $\\text{SU}(N_c)$ Yang-Mills theory\nin $d=2$ dimensions if $N_b$ is larger than $N_c-\\frac{3}{4}$ and conjectured,\nfollowing the argument of Budzcies and Zirnbauer, that this remains true for\n$d>2$. In the present paper, we test this conjecture by performing lattice\nsimulations of the simplest nontrivial case, i.e., gauge group $\\text{SU}(2)$\nin three dimensions. We show that observables computed in the ind

In [62]:
recommend("1212.3616",30) #pure tf-idf results

 
 1) Implications of the AdS/CFT Correspondence on Spacetime and Worldsheet
  OPE Coefficients by Sudip Ghosh, Sourav Sarkar and Mritunjay Verma (0.74077374). Arxiv: 1703.06132
 
 2) Recursive construction of the operator product expansion in curved space by Markus B Fr\"ob (0.725186). Arxiv: 2007.15668
 
 3) The Operator Product Expansion between the 16 Lowest Higher Spin
  Currents in the N=4 Superspace by Changhyun Ahn and Man Hea Kim (0.70562214). Arxiv: 1509.01908
 
 4) Holographic OPE Coefficients from AdS Black Holes with Matters by Yue-Zhou Li, Zhan-Feng Mai, H. Lu (0.7030324). Arxiv: 1905.09302
 
 5) Solving M-theory with the Conformal Bootstrap by Nathan B. Agmon, Shai M. Chester, Silviu S. Pufu (0.6985825). Arxiv: 1711.07343
 
 6) Analytic bootstrap at large spin by Apratim Kaviraj, Kallol Sen and Aninda Sinha (0.69455445). Arxiv: 1502.01437
 
 7) Random Statistics of OPE Coefficients and Euclidean Wormholes by Alexandre Belin, Jan de Boer (0.6894057). Arxiv: 2006.05499
 
 

In [70]:
fast_gpt_tfidf_recommender(sentence_model_e2,"1703.00278")


 1). With TFIDF rank 0.10662900656461716 and GPT-2 rank 2 
1) Paper title : ['Polyakov formulas for GJMS operators from AdS/CFT']. 
 Abstract : ['  We argue that the AdS/CFT calculational prescription for double-trace\ndeformations leads to a holographic derivation of the conformal anomaly, and\nits conformal primitive, associated to the whole family of conformally\ncovariant powers of the Laplacian (GJMS operators) at the conformal boundary.\nThe bulk side involves a quantum 1-loop correction to the SUGRA action and the\nboundary counterpart accounts for a sub-leading term in the large-N limit. The\nsequence of GJMS conformal Laplacians shows up in the two-point function of the\nCFT operator dual to a bulk scalar field at certain values of its scaling\ndimension. The restriction to conformally flat boundary metrics reduces the\nbulk computation to that of volume renormalization which renders the universal\ntype A anomaly. In this way, we directly connect two chief roles of the\nQ-cur


 7). With TFIDF rank 0.11777441203594208 and GPT-2 rank 25 
7) Paper title : ['Real-time finite-temperature correlators from AdS/CFT']. 
 Abstract : ["  In this paper we use AdS/CFT ideas in conjunction with insights from finite\ntemperature real-time field theory formalism to compute 3-point correlators of\n${\\cal N}{=}4$ super Yang-Mills operators, in real time and at finite\ntemperature. To this end, we propose that the gravity field action is\nintegrated only over the right and left quadrants of the Penrose diagram of the\nAnti de Sitter-Schwarzschild background, with a relative sign between the two\nterms. For concreteness we consider the case of a scalar field in the black\nhole background. Using the scalar field Schwinger-Keldysh bulk-to-boundary\npropagators, we give the general expression of a 3-point real-time Green's\ncorrelator. We then note that this particular prescription amounts to adapting\nthe finite-temperature analog of Veltman's circling rules to tree-level Witte


 12). With TFIDF rank 0.11869436502456665 and GPT-2 rank 40 
12) Paper title : ['The Ryu-Takayanagi Formula from Quantum Error Correction']. 
 Abstract : ['  I argue that a version of the quantum-corrected Ryu-Takayanagi formula holds\nin any quantum error-correcting code. I present this result as a series of\ntheorems of increasing generality, with the final statement expressed in the\nlanguage of operator-algebra quantum error correction. In AdS/CFT this gives a\n"purely boundary" interpretation of the formula. I also extend a recent\ntheorem, which established entanglement-wedge reconstruction in AdS/CFT, when\ninterpreted as a subsystem code, to the more general, and I argue more\nphysical, case of subalgebra codes. For completeness, I include a\nself-contained presentation of the theory of von Neumann algebras on\nfinite-dimensional Hilbert spaces, as well as the algebraic definition of\nentropy. The results confirm a close relationship between bulk gauge\ntransformations, edge-m


 17). With TFIDF rank 0.12236008793115616 and GPT-2 rank 54 
17) Paper title : ['Numerical study of the Lorentzian Engle-Pereira-Rovelli-Livine spin foam\n  amplitude']. 
 Abstract : ['  The Lorentzian EPRL spin foam amplitude for loop quantum gravity is a\nmulti-dimensional non-compact integral of highly oscillating functions. Using a\nmethod based on the decomposition of Clebsch-Gordan coefficients for the\nunitary infinite-dimensional representations of SL(2,C) in terms of those of\nSU(2), we are able to provide for the first time numerical evaluations of the\nvertex amplitude. The values obtained support the asymptotic formula obtained\nby Barrett and collaborators with a saddle point approximation, showing, in\nparticular, a power-law decay and oscillations related to the Regge action. The\ncomparison offers a test of the efficiency of the method. Truncating the\ndecomposition to the first few terms provides a qualitative matching of the\npower-law decay and oscillations. For vec


 22). With TFIDF rank 0.10977703332901001 and GPT-2 rank 87 
22) Paper title : ['Double Trace Flows and Holographic RG in dS/CFT correspondence']. 
 Abstract : ["  If there is a dS/CFT correspondence, time evolution in the bulk should\ntranslate to RG flows in the dual euclidean field theory. Consequently,\nalthough the dual field is expected to be non-unitary, its RG flows will carry\nan imprint of the unitary time evolution in the bulk. In this note we examine\nthe prediction of holographic RG in de Sitter space for the flow of double and\ntriple trace couplings in any proposed dual. We show quite generally that the\ncorrect form of the field theory beta functions for the double trace couplings\nis obtained from holography, provided one identifies the scale of the field\ntheory with (i|T|) where T is the `time' in conformal coordinates. For dS(4),\nwe find that with an appropriate choice of operator normalization, it is\npossible to have real n-point correlation functions as well as


 27). With TFIDF rank 0.11537638306617737 and GPT-2 rank 110 
27) Paper title : ['Subleading Regge limit from a soft anomalous dimension']. 
 Abstract : ['  Wilson lines capture important features of scattering amplitudes, for example\nsoft effects relevant for infrared divergences, and the Regge limit. Beyond the\nleading power approximation, corrections to the eikonal picture have to be\ntaken into account. In this paper, we study such corrections in a model of\nmassive scattering amplitudes in N = 4 super Yang-Mills, in the planar limit,\nwhere the mass is generated through a Higgs mechanism. Using known three-loop\nanalytic expressions for the scattering amplitude, we find that the first power\nsuppressed term has a very simple form, equal to a single power law. We propose\nthat its exponent is governed by the anomalous dimension of a Wilson loop with\na scalar inserted at the cusp, and we provide perturbative evidence for this\nproposal. We also analyze other limits of the amplit